In [2]:
# Loading the packages to be used
from __future__ import print_function  # Python 2 and 3
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
from scipy import stats
import math
%matplotlib inline

In [8]:
# Import data; for linear regression, either import encoded datasets or dummify and then oneHotEncode in python
houses_train = pd.read_csv('../Data/Xencoded_houses_train.csv')
houses_test = pd.read_csv('../Data/Xencoded_houses_test.csv')
private_train = pd.read_csv('../Data/Xencoded_private_train.csv')
private_test = pd.read_csv('../Data/Xencoded_private_test.csv')
#private_test_encoded = pd.read_csv('../Data/Xencoded_private_test.csv')


In [26]:
X_private_train.shape

(1168, 216)

In [9]:
# Define data frame of predictors and BoxCox response variable
X_houses_train = houses_train.drop('SalePrice', axis = 1)
houses_trainBC, houses_lambda = stats.boxcox(houses_train[['SalePrice']]) # T indicates transformation; ndarray output

X_private_train = private_train.drop('SalePrice', axis = 1)
print(X_private_train)
X_private_train=X_private_train.values
private_trainBC, private_lambda = stats.boxcox(private_train[['SalePrice']]) # T indicates transformation; ndarray output

private_test_encoded= private_test_encoded.drop('SalePrice', axis=1)

      MSSubClass_lev_x.120  MSSubClass_lev_x.160  MSSubClass_lev_x.190  \
0                 0.000000               0.00000                   0.0   
1                 0.000000               0.00000                   0.0   
2                 0.000000               0.00000                   0.0   
3                 0.000000               0.00000                   0.0   
4                 0.000000               0.00000                   0.0   
5                 0.000000               0.00000                   0.0   
6                 4.142676               0.00000                   0.0   
7                 0.000000               0.00000                   0.0   
8                 0.000000               4.60632                   0.0   
9                 0.000000               0.00000                   0.0   
10                0.000000               0.00000                   0.0   
11                0.000000               0.00000                   0.0   
12                0.000000            

NameError: name 'private_test_encoded' is not defined

In [10]:
# Turn ndarray of BoxCoxed response variable into a data frame
yT_houses_train = pd.DataFrame(houses_trainBC)
yT_private_train = pd.DataFrame(private_trainBC)

yT_houses_train.columns = ['SalePrice_BC']
yT_private_train.columns = ['SalePrice_BC']

private_test[['SalePrice']]= np.log(private_test[['SalePrice']].values)




In [11]:
#cross validate elastic net regularization for parameters alpha
from sklearn.metrics import mean_squared_error as mse
from sklearn.linear_model import ElasticNetCV


yT_private_train=np.ravel(yT_private_train)

alphas = np.linspace(0.01,1,20)
l1_params= np.linspace(0.01,1,20)

ENCV= ElasticNetCV(alphas= alphas,l1_ratio= l1_params, n_alphas= 1000,cv=5, random_state=0)

ENCV.fit(X_private_train, yT_private_train)

predictions_train=ENCV.predict(private_test_encoded)


#transform predictions boxcox to real values
predictions_transformed= np.log(np.power((predictions_train * private_lambda) + 1, 1 / private_lambda))


#rmse for test data
rmse_test=math.sqrt(mse(predictions_transformed,private_test[['SalePrice']]))


#optimum alpha value and l1 ratio value
print('rmse_test alpha\t l1_ratio score')
print("{:0.5f}\t {:0.4f}\t {:0.4f}\t {:0.4f}".format(rmse_test, ENCV.alpha_, ENCV.l1_ratio_, ENCV.score(X_private_train, yT_private_train)))




NameError: name 'private_test_encoded' is not defined

In [282]:
from sklearn.linear_model import Lasso, Ridge, ElasticNet
from sklearn.metrics import mean_squared_error as mse
from sklearn.cross_validation import KFold

t_rmse = np.array([])
cv_rmse = np.array([])
yT_private_train=np.ravel(yT_private_train)

alphas = np.linspace(0.01,1,30)



print('model\t alpha\t RMSE_train\t RMSE_5cv \t R square\n')

for a in alphas:
    models=[('lasso', Lasso( fit_intercept=True, alpha=a)),
            ('ridge', Ridge(fit_intercept=True, alpha=a)),
            ('el_net', ElasticNet(fit_intercept=True, alpha=a))]
    
    
    print('\n')
    for name, method in models:
              
        
        method.fit(X_private_train, yT_private_train)
        
        score= method.score(X_private_train, yT_private_train)
    
        # computing the RMSE on training data
        
        p = method.predict(X_private_train)
        err = p- yT_private_train
        total_error_squared = np.dot(err,err)
        rmse_train = np.sqrt(total_error_squared/len(p))

        # computing RMSE using 5-fold cross validation
        kf = KFold(len(X_private_train), n_folds=5, random_state=0)

        xval_err_squared = 0
        for train, test in kf:
            method.fit(X_private_train[train], yT_private_train[train])
            p = method.predict(X_private_train[test])
            
            err = p - yT_private_train[test]
            xval_err_squared += np.dot(err,err)
        rmse_5cv = np.sqrt(xval_err_squared/len(X_private_train))

        t_rmse = np.append(t_rmse, [rmse_train])
        cv_rmse = np.append(cv_rmse, [rmse_5cv])
        
        
        print('{}\t {:0.3f}\t {:.5f}\t {:.6f}\t{:.5f}'.format(name,a,rmse_train,rmse_5cv, score))

        #print('%s \t %0.3f \t %.5f \t %.6f \t %.5f'%(name,a,rmse_train,rmse_5cv, score))



model	 alpha	 RMSE_train	 RMSE_5cv 	 R square



lasso	 0.010	 0.06410	 0.078191	0.88654
ridge	 0.010	 0.03995	 0.081838	0.95593
el_net	 0.010	 0.05337	 0.070400	0.92135


lasso	 0.044	 0.10462	 0.107258	0.69775
ridge	 0.044	 0.03995	 0.081816	0.95593
el_net	 0.044	 0.08461	 0.089964	0.80232


lasso	 0.078	 0.13757	 0.138651	0.47739
ridge	 0.078	 0.03995	 0.081794	0.95593
el_net	 0.078	 0.10080	 0.103403	0.71943


lasso	 0.112	 0.17058	 0.171435	0.19657
ridge	 0.112	 0.03995	 0.081772	0.95593
el_net	 0.112	 0.11627	 0.118146	0.62670


lasso	 0.147	 0.19030	 0.190418	0.00000
ridge	 0.147	 0.03995	 0.081751	0.95593
el_net	 0.147	 0.13322	 0.134492	0.50995


lasso	 0.181	 0.19030	 0.190418	0.00000
ridge	 0.181	 0.03995	 0.081730	0.95593
el_net	 0.181	 0.15084	 0.151551	0.37173


lasso	 0.215	 0.19030	 0.190418	0.00000
ridge	 0.215	 0.03995	 0.081710	0.95593
el_net	 0.215	 0.16661	 0.167397	0.23347


lasso	 0.249	 0.19030	 0.190418	0.00000
ridge	 0.249	 0.03995	 0.081689	0.95593
el_net	 0.

In [15]:
from sklearn.linear_model import Lasso, Ridge, ElasticNet
from sklearn.metrics import mean_squared_error as mse
from sklearn.cross_validation import KFold

values= np.array([])


yT_private_train=np.ravel(yT_private_train)

alphas = np.linspace(0.01,1,30)

l1_params= np.linspace(0.01,1,20)

print('model\t l1\t alpha\t RMSE_train\t RMSE_5cv \t R square\n')
print('\n')
for l1 in l1_params:  
    
                      
    for a in alphas:
        EN= ElasticNet(fit_intercept= True, alpha=a, l1_ratio= l1, random_state=0)
        
        EN.fit(X_private_train, yT_private_train)
        
        score= EN.score(X_private_train, yT_private_train)
    
        # computing the RMSE on training data
        
        p = EN.predict(X_private_train)
        err = p- yT_private_train
        total_error_squared = np.dot(err,err)
        rmse_train = np.sqrt(total_error_squared/len(p))

        # computing RMSE using 5-fold cross validation
        kf = KFold(len(X_private_train), n_folds=5, random_state=0)

        xval_err_squared = 0
        for train, test in kf:
            EN.fit(X_private_train[train], yT_private_train[train])
            p = EN.predict(X_private_train[test])
            err = p - yT_private_train[test]
            xval_err_squared += np.dot(err,err)
        rmse_5cv = np.sqrt(xval_err_squared/len(X_private_train))

        values = np.append(values,[l1,a,rmse_train, rmse_5cv, score])
        
        
        print('{:0.3f}\t {:0.3f}\t {:.5f}\t {:.6f}\t{:.5f}'.format(l1,a,rmse_train,rmse_5cv, score))

        #print('%s \t %0.3f \t %.5f \t %.6f \t %.5f'%(name,a,rmse_train,rmse_5cv, score))

model	 l1	 alpha	 RMSE_train	 RMSE_5cv 	 R square



0.010	 0.010	 0.05252	 0.071323	0.92382
0.010	 0.044	 0.05760	 0.070838	0.90839
0.010	 0.078	 0.06049	 0.071272	0.89898
0.010	 0.112	 0.06264	 0.072002	0.89165
0.010	 0.147	 0.06449	 0.072919	0.88516
0.010	 0.181	 0.06614	 0.073795	0.87921
0.010	 0.215	 0.06761	 0.074691	0.87379
0.010	 0.249	 0.06898	 0.075598	0.86861
0.010	 0.283	 0.07030	 0.076505	0.86354
0.010	 0.317	 0.07153	 0.077405	0.85874
0.010	 0.351	 0.07269	 0.078308	0.85411
0.010	 0.386	 0.07382	 0.079217	0.84951
0.010	 0.420	 0.07494	 0.080120	0.84491
0.010	 0.454	 0.07603	 0.081036	0.84037
0.010	 0.488	 0.07709	 0.081935	0.83592
0.010	 0.522	 0.07813	 0.082828	0.83145
0.010	 0.556	 0.07913	 0.083715	0.82708
0.010	 0.590	 0.08013	 0.084596	0.82269
0.010	 0.624	 0.08109	 0.085478	0.81841
0.010	 0.659	 0.08204	 0.086364	0.81413
0.010	 0.693	 0.08299	 0.087244	0.80982
0.010	 0.727	 0.08394	 0.088118	0.80546
0.010	 0.761	 0.08488	 0.088978	0.80106
0.010	 0.795	 0.08582	 0.08

0.323	 0.863	 0.19030	 0.190418	0.00000
0.323	 0.898	 0.19030	 0.190418	0.00000
0.323	 0.932	 0.19030	 0.190418	0.00000
0.323	 0.966	 0.19030	 0.190418	0.00000
0.323	 1.000	 0.19030	 0.190418	0.00000
0.375	 0.010	 0.06458	 0.073213	0.88485
0.375	 0.044	 0.09536	 0.098180	0.74888
0.375	 0.078	 0.11578	 0.117331	0.62984
0.375	 0.112	 0.13233	 0.133143	0.51647
0.375	 0.147	 0.14133	 0.141562	0.44847
0.375	 0.181	 0.14753	 0.147806	0.39901
0.375	 0.215	 0.15430	 0.154585	0.34257
0.375	 0.249	 0.16148	 0.161767	0.28001
0.375	 0.283	 0.16892	 0.169216	0.21211
0.375	 0.317	 0.17653	 0.176829	0.13955
0.375	 0.351	 0.18422	 0.184525	0.06294
0.375	 0.386	 0.19030	 0.190387	0.00000
0.375	 0.420	 0.19030	 0.190418	0.00000
0.375	 0.454	 0.19030	 0.190418	0.00000
0.375	 0.488	 0.19030	 0.190418	0.00000
0.375	 0.522	 0.19030	 0.190418	0.00000
0.375	 0.556	 0.19030	 0.190418	0.00000
0.375	 0.590	 0.19030	 0.190418	0.00000
0.375	 0.624	 0.19030	 0.190418	0.00000
0.375	 0.659	 0.19030	 0.190418	0.00000


0.687	 0.761	 0.19030	 0.190418	0.00000
0.687	 0.795	 0.19030	 0.190418	0.00000
0.687	 0.829	 0.19030	 0.190418	0.00000
0.687	 0.863	 0.19030	 0.190418	0.00000
0.687	 0.898	 0.19030	 0.190418	0.00000
0.687	 0.932	 0.19030	 0.190418	0.00000
0.687	 0.966	 0.19030	 0.190418	0.00000
0.687	 1.000	 0.19030	 0.190418	0.00000
0.739	 0.010	 0.07491	 0.081310	0.84506
0.739	 0.044	 0.12073	 0.122106	0.59751
0.739	 0.078	 0.14018	 0.140460	0.45740
0.739	 0.112	 0.15263	 0.152935	0.35672
0.739	 0.147	 0.16770	 0.168023	0.22346
0.739	 0.181	 0.18457	 0.184918	0.05933
0.739	 0.215	 0.19030	 0.190418	0.00000
0.739	 0.249	 0.19030	 0.190418	0.00000
0.739	 0.283	 0.19030	 0.190418	0.00000
0.739	 0.317	 0.19030	 0.190418	0.00000
0.739	 0.351	 0.19030	 0.190418	0.00000
0.739	 0.386	 0.19030	 0.190418	0.00000
0.739	 0.420	 0.19030	 0.190418	0.00000
0.739	 0.454	 0.19030	 0.190418	0.00000
0.739	 0.488	 0.19030	 0.190418	0.00000
0.739	 0.522	 0.19030	 0.190418	0.00000
0.739	 0.556	 0.19030	 0.190418	0.00000


In [16]:
df_EN=pd.DataFrame(values.reshape(600,5), columns= ["l1", "alpha", "rmse_train", "rmse_5cv", "score"])

df_EN[df_EN["rmse_5cv"]==min(df_EN["rmse_5cv"])]

,l1,alpha,rmse_train,rmse_5cv,score
30,0.062105,0.01,0.054864,0.069995,0.916885


In [ ]:
#best parameters l1=0.01 alpha= 0.18069

In [20]:
X_private_train.shape

(1168, 216)

In [21]:
yT_private_train.shape

(1168,)

In [22]:
p.shape

(233,)

In [25]:
private_test[['SalePrice']].shape

(292, 1)

In [19]:
EN_tuned= ElasticNet(fit_intercept= True, alpha=0.18069, l1_ratio= 0.01, random_state=0)
        
EN_tuned.fit(X_private_train, yT_private_train)
        
score= EN_tuned.score(X_private_train, yT_private_train)
    
# computing the RMSE on training data
        
p = EN_tuned.predict(private_test)
p_transformed=np.log(np.power((p * private_lambda) + 1, 1 / private_lambda))
#rmse_test=math.sqrt(mse(p_transformed,private_test[['SalePrice']]))

print(rmse_test,score)

ValueError: shapes (292,217) and (216,) not aligned: 217 (dim 1) != 216 (dim 0)

In [24]:
ridge_tuned= Ridge(fit_intercept= True, alpha=1, random_state=0)
        
ridge_tuned.fit(X_private_train, yT_private_train)
        
score= ridge_tuned.score(X_private_train, yT_private_train)
    
# computing the RMSE on training data
        
p = ridge_tuned.predict(private_test)
p_transformed=np.log(np.power((p * private_lambda) + 1, 1 / private_lambda))
rmse_test=math.sqrt(mse(p_transformed,private_test[['SalePrice']]))

print(rmse_test, score)

ValueError: shapes (292,217) and (216,) not aligned: 217 (dim 1) != 216 (dim 0)

In [326]:
lasso_tuned= Lasso(fit_intercept= True, alpha=0.01, random_state=0)
        
lasso_tuned.fit(X_private_train, yT_private_train)
        
score= lasso_tuned.score(X_private_train, yT_private_train)
    
# computing the RMSE on training data
        
p = lasso_tuned.predict(private_test_encoded)
p_transformed=np.log(np.power((p * private_lambda) + 1, 1 / private_lambda))
rmse_test=math.sqrt(mse(p_transformed,private_test[['SalePrice']]))

print(rmse_test, score)

0.16101903616756894 0.886543337683
